In [1]:
import json
import ollama
import re

In [2]:
# Agent 7: Ethics & Risk Checker
# Reads project ideas and flags bias, misuse risk, privacy, and explainability issues.
# Provides a risk level (low, medium, high) for each and mitigation suggestions.
INPUT_FILE = 'output/agent2_ideas.json'
OUTPUT_FILE = 'output/agent7_risks.json'

# LLM_MODEL = 'llama3.2:3b'
AGENT_7 = 'qwen3:32b'

In [3]:
PROMPT_TEMPLATE = """
You are Agent 7: an Ethics & Risk Checker for climate-change AI projects.

Project idea:
Title: {title}
Description: {description}
ID: {idea_id}

Considering the following potential risks in AI systems:
- BIAS: Does the system risk reinforcing existing inequities or disadvantaging certain groups?
- MISUSE: Could bad actors exploit this system for harmful purposes?
- PRIVACY: Does the system collect, process, or generate sensitive data about individuals or groups?
- EXPLAINABILITY: Are the system's decisions transparent and interpretable to users and stakeholders?
- ECOLOGICAL IMPACT: Could the system directly or indirectly cause environmental harm?

First, identify the TWO MOST SIGNIFICANT risks specific to this particular project.
Then, assign an overall risk level (low, medium, or high) based on your analysis.
Finally, provide 3 HIGHLY SPECIFIC mitigation suggestions that directly address the identified risks for THIS PROJECT.

RESPONSE FORMAT:
Return your response in this exact format:

IDEA_ID: {idea_id}
RISK_LEVEL: [low/medium/high]
MITIGATION_SUGGESTIONS:
1. [First project-specific mitigation suggestion that addresses a concrete risk]
2. [Second project-specific mitigation suggestion with technical or procedural detail]
3. [Third project-specific mitigation suggestion with measurable outcomes]

Your mitigation suggestions must be SPECIFIC to this project, ACTIONABLE, and DIVERSE. Do not use generic solutions that could apply to any AI system. Each suggestion should be substantially different from the otherGs.
"""

In [4]:
def load_ideas(path: str) -> list:
    """Load project ideas from JSON produced by Agent 2."""
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [5]:
def extract_ethics_check(text: str) -> dict:
    """Extract ethics and risk check information from the LLM response."""
    # Clean up the text
    text = text.replace('```', '').strip()

    # Initialize variables
    idea_id = None
    risk_level = None
    mitigation_suggestions = []

    # Keep track of section we're processing
    in_suggestions = False

    # Process line by line
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue

        # Extract idea ID
        if line.startswith("IDEA_ID:"):
            idea_id = line[len("IDEA_ID:"):].strip()

        # Extract risk level
        elif line.startswith("RISK_LEVEL:"):
            risk_value = line[len("RISK_LEVEL:"):].strip().lower()
            # Normalize risk value to one of the expected values
            if "low" in risk_value:
                risk_level = "low"
            elif "medium" in risk_value or "med" in risk_value:
                risk_level = "medium"
            elif "high" in risk_value:
                risk_level = "high"
            else:
                # Default if unable to determine
                risk_level = "medium"

        # Track when we're in the mitigation suggestions section
        elif line.startswith("MITIGATION_SUGGESTIONS:"):
            in_suggestions = True

        # Extract numbered suggestions
        elif in_suggestions and (line.startswith("1.") or line.startswith("2.") or
                                line.startswith("3.") or re.match(r"^\d+\.", line)):
            # Extract the suggestion text without the number prefix
            suggestion = re.sub(r"^\d+\.\s*", "", line).strip()
            if suggestion:
                mitigation_suggestions.append(suggestion)

        # Handle suggestions that might not be numbered
        elif in_suggestions and not any(line.startswith(prefix) for prefix in
                                     ["IDEA_ID:", "RISK_LEVEL:", "MITIGATION_SUGGESTIONS:"]):
            # This could be a continuation of suggestions or an unmarked suggestion
            if not mitigation_suggestions or len(mitigation_suggestions) >= 3:
                # Start a new suggestion if we have none or already have 3+
                mitigation_suggestions.append(line)
            else:
                # Might be a continuation of the last suggestion
                mitigation_suggestions[-1] += " " + line

    # If we found fewer than 3 suggestions but found some text after MITIGATION_SUGGESTIONS,
    # try to split it into separate suggestions
    if in_suggestions and len(mitigation_suggestions) < 3:
        # Look for sentences that could be separate suggestions
        extra_text = " ".join(mitigation_suggestions)
        sentences = re.findall(r'[^.!?]+[.!?]', extra_text)
        if len(sentences) >= 3:
            mitigation_suggestions = [s.strip() for s in sentences[:3]]

    # Ensure we have exactly 3 suggestions
    while len(mitigation_suggestions) < 3:
        mitigation_suggestions.append("Ensure regular ethical review of the project.")

    # Truncate to 3 if we found more
    mitigation_suggestions = mitigation_suggestions[:3]

    # Use default values if parsing failed
    if not idea_id:
        idea_id = "unknown_id"

    if not risk_level:
        risk_level = "medium"

    # Return as a dictionary in the expected format
    return {
        "idea_id": idea_id,
        "risk_level": risk_level,
        "mitigation_suggestions": mitigation_suggestions
    }

In [6]:
def assess_risk(idea: dict) -> dict:
    """Assess overall risk and mitigation suggestions for a single idea."""
    # Format the prompt with all necessary fields
    prompt = PROMPT_TEMPLATE.format(
        title=idea['title'],
        description=idea['description'],
        idea_id=idea.get('idea_id', '')
    )

    print(f"Assessing risks for idea: {idea.get('idea_id', '')}")

    try:
        # Try chat API first
        response = ollama.chat(
            model=AGENT_7,  # Make sure AGENT_7 is defined
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        text = response['message']['content'] if 'message' in response else str(response)
    except Exception as e:
        print(f"Chat API error: {e}, falling back to generate")
        try:
            # Fallback to generate API
            response = ollama.generate(model=AGENT_7, prompt=prompt)
            if hasattr(response, 'message'):
                text = response.message.content
            elif isinstance(response, dict) and 'response' in response:
                text = response['response']
            else:
                text = str(response)
        except Exception as e2:
            print(f"Generate API error: {e2}")
            text = f"Error calling LLM: {e2}"

    # Save raw response for debugging
    debug_file = f"output/debug/risk_debug_{idea.get('idea_id', 'unknown')}.txt"
    with open(debug_file, "w") as f:
        f.write(text)

    print(f"Raw response saved to {debug_file}")

    try:
        # Use our new parsing function
        result = extract_ethics_check(text)

        # Log what we found
        print(f"Parsed risk level: {result['risk_level']}")
        print(f"Found {len(result['mitigation_suggestions'])} mitigation suggestions")

        # Ensure idea_id is correct
        if result['idea_id'] != idea.get('idea_id', ''):
            print(f"Warning: Extracted idea_id '{result['idea_id']}' doesn't match expected '{idea.get('idea_id', '')}'")
            result['idea_id'] = idea.get('idea_id', '')

    except Exception as e:
        print(f"Error parsing response: {e}")
        # Provide default values if parsing fails
        result = {
            "idea_id": idea.get('idea_id', ''),
            "risk_level": "medium",
            "mitigation_suggestions": [
                "Ensure transparent documentation of model limitations and potential biases.",
                "Implement robust data governance and privacy protection measures.",
                "Establish regular ethical review processes throughout development and deployment."
            ]
        }
        print("Using default risk assessment due to parsing error")

    return result

In [7]:
def run_agent7():
    """Run ethics & risk checker on all ideas and save results."""
    print("\n=== Starting Agent 7: Ethics & Risk Checker ===")

    try:
        # Load ideas
        ideas = load_ideas(INPUT_FILE)
        print(f"Loaded {len(ideas)} ideas from {INPUT_FILE}")

        outputs = []

        # Process each idea with progress tracking
        for i, idea in enumerate(ideas):
            print(f"\nProcessing idea {i+1}/{len(ideas)}: {idea.get('title', 'Untitled')}")
            try:
                assessment = assess_risk(idea)
                outputs.append(assessment)

                # Print a summary of the assessment
                risk_level = assessment.get('risk_level', 'unknown')
                risk_color = {
                    'low': '\033[92m',    # Green
                    'medium': '\033[93m',  # Yellow
                    'high': '\033[91m'     # Red
                }.get(risk_level.lower(), '')
                reset_color = '\033[0m'

                print(f"Risk level: {risk_color}{risk_level}{reset_color}")
                print("Mitigation suggestions:")
                for j, suggestion in enumerate(assessment.get('mitigation_suggestions', []), 1):
                    print(f"  {j}. {suggestion}")

            except Exception as e:
                print(f"ERROR processing idea {idea.get('idea_id', '')}: {e}")
                # Create a default assessment to avoid breaking the pipeline
                outputs.append({
                    "idea_id": idea.get('idea_id', f"unknown_{i}"),
                    "risk_level": "medium",
                    "mitigation_suggestions": [
                        "Implement robust data governance policies.",
                        "Ensure transparent documentation of model limitations.",
                        "Establish regular ethical review processes."
                    ]
                })

        # Write results to file
        print(f"\nWriting {len(outputs)} risk assessments to {OUTPUT_FILE}")
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            json.dump(outputs, f, ensure_ascii=False, indent=2)

        # Generate summary statistics
        risk_counts = {
            'low': 0,
            'medium': 0,
            'high': 0
        }

        for output in outputs:
            risk = output.get('risk_level', '').lower()
            if risk in risk_counts:
                risk_counts[risk] += 1

        print("\n=== Risk Assessment Summary ===")
        print(f"Total ideas processed: {len(outputs)}")
        print(f"Risk levels: {risk_counts['low']} low, {risk_counts['medium']} medium, {risk_counts['high']} high")
        print(f"Results saved to {OUTPUT_FILE}")
        print("=== Agent 7 completed successfully ===\n")

    except Exception as e:
        print(f"CRITICAL ERROR in Agent 7: {e}")
        print("Agent 7 failed to complete")

if __name__ == "__main__":
    run_agent7()


=== Starting Agent 7: Ethics & Risk Checker ===
Loaded 20 ideas from output/agent2_ideas.json

Processing idea 1/20: Arctic Anomaly Detection with Computer Vision
Assessing risks for idea: 8_arctic_anomaly_detection_with_computer_vision
Raw response saved to output/debug/risk_debug_8_arctic_anomaly_detection_with_computer_vision.txt
Parsed risk level: medium
Found 3 mitigation suggestions
Risk level: medium
Mitigation suggestions:
  1. **Implement region-specific bias audits** by validating model performance across all Arctic sub-regions (e.g., Greenland, Siberia, Canadian Arctic) using stratified sampling during training and testing to ensure equitable detection accuracy.
  2. **Integrate model-agnostic explanation tools** (e.g., SHAP or LIME) to generate interpretable heatmaps of CNN outputs, highlighting specific spectral bands and spatial features contributing to anomaly classifications for stakeholders.
  3. **Develop a feedback loop with Arctic Indigenous communities** to ground

In [14]:
import shutil
from pathlib import Path

# 1) Source and destination directories
source_dir = Path('./output')
dest_dir   = Path('./output/3IdeaPasses')

# 2) Ensure the destination exists
dest_dir.mkdir(parents=True, exist_ok=True)

# 3) List of filenames to move
files = [
    'agent2_ideas.json',
    'agent2_raw_llm_output.txt',
    'task4.json',
    'agent5_co2.json',
    'agent6_impact.json',
    'agent7_risks.json'
]

# 4) Move each one
for fname in files:
    src = source_dir / fname
    dst = dest_dir   / fname
    if src.exists():
        shutil.copy(str(src), str(dst))
        print(f"Copied {src} → {dst}")
    else:
        print(f"{src} not found, skipping.")

Copied output/agent2_ideas.json → output/3IdeaPasses/agent2_ideas.json
Copied output/agent2_raw_llm_output.txt → output/3IdeaPasses/agent2_raw_llm_output.txt
Copied output/agent4.json → output/3IdeaPasses/agent4.json
Copied output/task4.json → output/3IdeaPasses/task4.json
Copied output/agent5_co2.json → output/3IdeaPasses/agent5_co2.json
Copied output/agent6_impact.json → output/3IdeaPasses/agent6_impact.json
Copied output/agent7_risks.json → output/3IdeaPasses/agent7_risks.json
